**MACHINE LEARNING - LAB ASSIGNMENT**

By: Madhumithaa RP | 20BCE1648 

Topic: **DECISION TREE IMPLEMENTATION**


# Steps for Decision Tree: 

What we need: 
1. Analyse the training dataset of to find the labels 
2. Find I(D) 
3. Find Info for each Attributes 
4. Substract both and calculate the Information gain(attr) For all attr in the dataset. 
5. Divide the dataset with the Attr with maximum Info Gain 
6. Divide the tree and do the same for all created branches 
7. Repeat these steps untill a stopping criteria is reached, like the maximum depth.

# Required Formulas: 

1. Information Gain, IG = E(Parent) - <weighted average>*E(Children)
2. Entropy, E = Sum(P(x)*log2(P(x)) where P(x) = #x/n 
3. Stopping Criteria = Maximum Depth 

# Importing required Libraries: 

In [32]:
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

# Dataset Overlook: 

In [54]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

In [56]:
print(data)

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]]), 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
 

# Node: 

We are creating the Node in order to construct the decision tree, as the tree contains left and right node, and the leave nodes at the bottom. 

In [33]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None



# Decision Tree Implementation 

__init__ defines the stop criteria of creating the decision tree. 

__grow tree__ will take the attr and target, it finds the best split and create child nodes - recursive function

__most common label__ counts the no of [0] and [1] 

__Information Gain__ will calculate the entropy of Parent 

__split__ will calculate the entropy of the other attr - basically left & right nodes. 

__best split__ takes attr & target, and it will compare the IGs and return the _best Attr_

In [28]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        



# Train & Test dataset

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)


# Training the dataset 

In [57]:
clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)


# Testing the tree

In [58]:
predictions = clf.predict(X_test)
print(predictions)

[1 1 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1
 0 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1
 1 1 1 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1
 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 0 1
 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 0
 0 0 1 0 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0
 1 0 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 1 1]


In [49]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)


In [50]:
acc = accuracy(y_test, predictions)
print(acc)

0.9157894736842105
